<img src="http://wandb.me/logo-im-png" width="400" alt="Weights & Biases" />
<!--- @wandbcode{python-report-api} -->

# 🔩 Setup

In [1]:
!pip install git+https://github.com/huggingface/transformers -qqq
!pip install datasets -qqq
!pip install bitsandbytes -qqq
!pip install huggingface_hub -qqq
!pip install peft -qqq
!pip install accelerate -qqq
!pip install bitsandbytes -qqq
!pip install wandb -qqq
!pip install einops -qqq

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.

In [2]:
import re
import os

from itertools import product
from tqdm import tqdm
from functools import partial
import random
import numpy as np
import torch
import accelerate
import bitsandbytes as bnb
import datasets

from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model,
    PeftModel
)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
!nvidia-smi

Tue Dec 19 05:24:14 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0              24W / 300W |      2MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# 🧠 Model

In [5]:
model_name = "microsoft/phi-2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",  # Auto selects device to put model on.
    trust_remote_code=True
)

model.config.use_cache = False
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=False)

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/24.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/577M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

In [6]:
# Used for LoRA.
def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    # lm_head is often excluded.
    if 'lm_head' in lora_module_names:  # needed for 16-bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)
modules = find_all_linear_names(model)

In [7]:
# Checking memory...
!nvidia-smi

Tue Dec 19 05:25:43 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              40W / 300W |   3168MiB / 16384MiB |      2%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# 📂 Dataset

In [8]:
dataset = load_dataset("Rowan/hellaswag")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/39905 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10003 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10042 [00:00<?, ? examples/s]

In [9]:
# Ref: https://github.com/EleutherAI/lm-evaluation-harness/blob/main/lm_eval/tasks/hellaswag/utils.py
def preprocess(text):
    text = text.strip()
    # NOTE: Brackets are artifacts of the WikiHow dataset portion of HellaSwag.
    text = text.replace(" [title]", ". ")
    text = re.sub("\\[.*?\\]", "", text)
    text = text.replace("  ", " ")
    return text

def process_doc(doc):
    ctx = doc["ctx_a"] + " " + doc["ctx_b"].capitalize()
    query = preprocess(doc["activity_label"] + ": " + ctx)
    choices = [preprocess(ending) for ending in doc["endings"]]
    gold = int(doc["label"])
    out_doc = {
        "query": query,
        "choices": choices,
        "gold": gold,
        "text": query + " " + choices[gold]
    }
    return out_doc

In [10]:
# Note: I'm training the model causally not through multiclass classification.
def preprocess_dataset(tokenizer, max_length, dataset, seed = 42):
    # Format each prompt.
    print("Preprocessing dataset...")
    dataset = dataset.map(process_doc)

    def preprocess_batch(batch, tokenizer, max_length):
        return tokenizer(
            batch["text"],
            max_length=max_length,
            truncation=True,
        )

    # Apply preprocessing to each batch of the dataset & and remove "conversations" and "text" fields.
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched=True,
    ).select_columns(['input_ids', 'attention_mask'])

    # Shuffle dataset.
    dataset = dataset.shuffle(seed=seed)

    return dataset

In [11]:
train_ds = preprocess_dataset(tokenizer, 750, dataset["train"])
validation_ds = preprocess_dataset(tokenizer, 750, dataset["validation"])

Preprocessing dataset...


Map:   0%|          | 0/39905 [00:00<?, ? examples/s]

Map:   0%|          | 0/39905 [00:00<?, ? examples/s]

Preprocessing dataset...


Map:   0%|          | 0/10042 [00:00<?, ? examples/s]

Map:   0%|          | 0/10042 [00:00<?, ? examples/s]

In [12]:
train_ds[0].keys()

dict_keys(['input_ids', 'attention_mask'])

In [13]:
mini_val_ds = validation_ds.select(list(range(100)))  # For test purposes & faster validation.

# 👾 Training

## Small Sweep Over Lora Config

In [14]:
training_args = TrainingArguments(
  output_dir="outputs",
  per_device_train_batch_size=8,
  per_device_eval_batch_size=8,
  gradient_accumulation_steps=4,
  learning_rate=2e-4,
  max_grad_norm=1.0,
  max_steps=100,
  lr_scheduler_type="linear",
  warmup_steps=20,
  fp16=True,
  logging_strategy="steps",
  logging_steps=1,
  save_strategy="steps",
  save_steps=10,
  optim="paged_adamw_8bit",
  evaluation_strategy="steps",
  eval_steps=10,
  report_to="wandb",
)

In [15]:
os.environ["WANDB_LOG_MODEL"] = "checkpoint"  # Log model checkpoints.
os.environ["WANDB_WATCH"] = "all"  # Log gradients and parameters.

lora_values = {
    "lora_alpha": [32],
    "lora_dropout": [0.1],
    "lora_r": [8, 16]
}

all_lora_comb = list(product(*lora_values.values()))

for cfg in all_lora_comb:
  run = wandb.init(
    project="cicd_and_wandb",
    name=f"lora-alpha{cfg[0]}_lora-dropout{cfg[1]}_lora-r{cfg[2]}",
    notes="""
    Notes:\n\n
    - tokenizer.pad_token = tokenizer.eos_token\n
    - model.config.use_cache = False\n
    - model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=False)\n
    """,
    config={
      "model_name": model_name,
      **bnb_config.to_diff_dict(),
      **training_args.to_dict(),
      "lora_alpha": cfg[0],
      "lora_dropout": cfg[1],
      "lora_r": cfg[2]
    }
  )

  peft_config = LoraConfig(
    lora_alpha=cfg[0],
    lora_dropout=cfg[1],
    target_modules=modules,
    r=cfg[2],
    bias="none",
    task_type="CAUSAL_LM"
  )

  model_peft = get_peft_model(model, peft_config)

  # For `PeftModel`s we can use `get_nb_trainable_parameters` to get the param counts.
  trainable, total = model_peft.get_nb_trainable_parameters()  # Returns a Tuple.
  print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

  trainer = Trainer(
    model=model_peft,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
    train_dataset=train_ds,
    eval_dataset=mini_val_ds,
  )

  trainer.train()
  run.finish()

wandb: Currently logged in as: vincenttu. Use `wandb login --relogin` to force relogin


Trainable: 10485760 | total: 2790169600 | Percentage: 0.3758%


You're using a CodeGenTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
10,2.666800,2.575754
20,2.181200,2.354582
30,2.189600,2.257211
40,2.144700,2.222050
50,2.188900,2.207612
60,2.179600,2.190583
70,2.168400,2.183701
80,2.077500,2.178495
90,2.117200,2.175805
100,2.328900,2.174025


wandb: Adding directory to artifact (./outputs/checkpoint-10)... Done. 0.2s
wandb: Adding directory to artifact (./outputs/checkpoint-20)... Done. 0.2s
wandb: Adding directory to artifact (./outputs/checkpoint-30)... Done. 0.3s
wandb: Adding directory to artifact (./outputs/checkpoint-40)... Done. 0.4s
wandb: Adding directory to artifact (./outputs/checkpoint-50)... Done. 0.2s
wandb: Adding directory to artifact (./outputs/checkpoint-60)... Done. 0.2s
wandb: Adding directory to artifact (./outputs/checkpoint-70)... Done. 0.3s
wandb: Adding directory to artifact (./outputs/checkpoint-80)... Done. 0.2s
wandb: Adding directory to artifact (./outputs/checkpoint-90)... Done. 0.2s
wandb: Adding directory to artifact (./outputs/checkpoint-100)... Done. 0.2s


eval/loss,█▄▂▂▂▁▁▁▁▁
eval/runtime,▁▅▁█▁▁▁▁▁▁
eval/samples_per_second,█▁█▁██▇██▇
eval/steps_per_second,█▁█▁██▇██▇
train/epoch,▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/learning_rate,▁▂▃▄▅▆▇▇███▇▇▇▇▆▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁
train/loss,▇▇▇█▇▆▄▅▅▄▄▃▄▃▂▂▄▂▄▃▃▂▂▃▂▁▃▂▂▂▁▂▃▂▂▃▂▄▂▄
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


Trainable: 20971520 | total: 2800655360 | Percentage: 0.7488%


Step,Training Loss,Validation Loss
10,2.669600,2.579509
20,2.181800,2.353456
30,2.193600,2.258352
40,2.144900,2.227311
50,2.189600,2.211235
60,2.184000,2.194244
70,2.167700,2.184687
80,2.077700,2.182460
90,2.120600,2.181161
100,2.327000,2.179456


Checkpoint destination directory outputs/checkpoint-10 already exists and is non-empty.Saving will proceed but saved results may be invalid.
wandb: Adding directory to artifact (./outputs/checkpoint-10)... Done. 0.4s
Checkpoint destination directory outputs/checkpoint-20 already exists and is non-empty.Saving will proceed but saved results may be invalid.
wandb: Adding directory to artifact (./outputs/checkpoint-20)... Done. 0.4s
Checkpoint destination directory outputs/checkpoint-30 already exists and is non-empty.Saving will proceed but saved results may be invalid.
wandb: Adding directory to artifact (./outputs/checkpoint-30)... Done. 0.4s
Checkpoint destination directory outputs/checkpoint-40 already exists and is non-empty.Saving will proceed but saved results may be invalid.
wandb: Adding directory to artifact (./outputs/checkpoint-40)... Done. 0.3s
Checkpoint destination directory outputs/checkpoint-50 already exists and is non-empty.Saving will proceed but saved results may be 

eval/loss,█▄▂▂▂▁▁▁▁▁
eval/runtime,▁▇▁█▁▁▁▁▁▁
eval/samples_per_second,█▁█▁██████
eval/steps_per_second,█▁█▁██████
train/epoch,▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/learning_rate,▁▂▃▄▅▆▇▇███▇▇▇▇▆▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁
train/loss,▇▇▇█▇▆▄▅▅▄▄▃▄▃▂▃▄▂▄▃▃▂▂▃▂▁▃▂▂▂▁▂▂▂▂▃▂▄▂▄
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


## Small Sweep Over TrainingArgument Schedulers

In [14]:
peft_config = LoraConfig(
  lora_alpha=16,
  lora_dropout=0.1,
  target_modules=modules,
  r=8,
  bias="none",
  task_type="CAUSAL_LM"
)

model_peft = get_peft_model(model, peft_config)

# For `PeftModel`s we can use `get_nb_trainable_parameters` to get the param counts.
trainable, total = model_peft.get_nb_trainable_parameters()  # Returns a Tuple.
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 10485760 | total: 2790169600 | Percentage: 0.3758%


In [15]:
for scheduler in ["cosine", "cosine_with_restarts", "reduce_lr_on_plateau"]:
  training_args = TrainingArguments(
    output_dir="outputs",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    max_grad_norm=1.0,
    max_steps=100,
    lr_scheduler_type=scheduler,
    warmup_steps=20,
    fp16=True,
    logging_strategy="steps",
    logging_steps=1,
    save_strategy="steps",
    save_steps=10,
    optim="paged_adamw_8bit",
    evaluation_strategy="steps",
    eval_steps=10,
    report_to="wandb",
  )

  run = wandb.init(
    project="cicd_and_wandb",
    name=f"scheduler_{scheduler}",
    notes="""
    Notes:\n\n
    - tokenizer.pad_token = tokenizer.eos_token\n
    - model.config.use_cache = False\n
    - model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=False)\n
    """,
    config={
      "model_name": model_name,
      **bnb_config.to_diff_dict(),
      **training_args.to_dict(),
      "lora_alpha": 16,
      "lora_dropout": 0.1,
      "lora_r": 8
    }
  )

  trainer = Trainer(
    model=model_peft,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
    train_dataset=train_ds,
    eval_dataset=mini_val_ds,
  )

  trainer.train()
  run.finish()

wandb: Currently logged in as: vincenttu. Use `wandb login --relogin` to force relogin


You're using a CodeGenTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
10,2.705800,2.630363
20,2.226200,2.377122
30,2.220500,2.285595
40,2.161700,2.238738
50,2.210200,2.216624
60,2.194600,2.199796
70,2.179000,2.193749
80,2.092100,2.190188
90,2.132700,2.188187
100,2.343200,2.187777


eval/loss,█▄▃▂▁▁▁▁▁▁
eval/runtime,▄▅▅▂▂▁▃█▆▁
eval/samples_per_second,▅▄▃▇▇▇▆▁▃█
eval/steps_per_second,▅▄▃▇▇█▆▁▃█
train/epoch,▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/learning_rate,▁▂▃▄▅▆▇▇█████▇▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▂▁▁▁▁▁▁
train/loss,▇▇▇██▆▅▅▅▄▅▃▄▃▂▂▄▂▄▃▃▂▂▃▂▁▃▂▂▂▁▂▂▂▂▃▂▄▂▄
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


Step,Training Loss,Validation Loss
10,2.178900,2.187573
20,1.929700,2.191075
30,1.999000,2.206534
40,1.989800,2.203204
50,2.089300,2.197824
60,2.111900,2.183944
70,2.118200,2.178090
80,2.048200,2.177250
90,2.097900,2.176641
100,2.309000,2.176304


Checkpoint destination directory outputs/checkpoint-10 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory outputs/checkpoint-20 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory outputs/checkpoint-30 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory outputs/checkpoint-40 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory outputs/checkpoint-50 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory outputs/checkpoint-60 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory outputs/checkpoint-70 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint de

eval/loss,▄▄█▇▆▃▁▁▁▁
eval/runtime,▇█▁▂▂▁▆█▂▂
eval/samples_per_second,▂▁█▇▇█▃▁▇▇
eval/steps_per_second,▂▁█▇▇█▃▁▇▇
train/epoch,▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/learning_rate,▁▂▃▄▅▆▇▇█████▇▇▇▇▇▆▆▆▅▅▅▄▄▄▃▃▃▂▂▂▂▁▁▁▁▁▁
train/loss,▃▄▃▆▅▃▂▃▄▃▄▁▃▂▁▂▄▁▅▃▄▂▁▄▂▁▄▃▄▃▁▄▄▃▄▅▄▇▃█
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


Step,Training Loss,Validation Loss
10,2.044900,2.209751
20,1.759900,2.232888
30,1.850300,2.254250
40,1.852300,2.246645
50,1.989500,2.219613
60,2.060100,2.196568
70,2.093100,2.192939
80,2.043300,2.193664
90,2.070500,2.188413
100,2.291300,2.177448


Checkpoint destination directory outputs/checkpoint-10 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory outputs/checkpoint-20 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory outputs/checkpoint-30 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory outputs/checkpoint-40 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory outputs/checkpoint-50 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory outputs/checkpoint-60 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory outputs/checkpoint-70 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint de

eval/loss,▄▆█▇▅▃▂▂▂▁
eval/runtime,▁▁█▂▁▁▁▁▁▂
eval/samples_per_second,▇█▁▇▇████▇
eval/steps_per_second,▇█▁▇▇████▇
train/epoch,▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,▄▄▂▅▄▂▂▂▃▂▂▁▂▂▁▂▃▂▅▃▄▂▂▄▃▃▅▄▅▄▃▅▆▄▆▆▅▇▅█
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


# 🚀 Optional: 10-shot Inference on Validation (first 100)

Though you don't need to run this section, I included it here to show how to run 10-shot inference on HellaSwag. phi-2 is a causal model and HellaSwag is a classification dataset, but you can still run phi-2 on HellaSwag. For every query, you have 4 possible choices (1 of them being correct). For every query, you can pass the query + a choice to the model for a score. So for every query, you have a list of 4 scores/losses. Simply pick the one with the lowest loss.

In [7]:
train_ds_process = dataset["train"].map(process_doc)
validation_ds_process = dataset["validation"].map(process_doc)

Map:   0%|          | 0/39905 [00:00<?, ? examples/s]

Map:   0%|          | 0/10042 [00:00<?, ? examples/s]

In [8]:
# Ref: https://github.com/EleutherAI/lm-evaluation-harness/blob/aa61f940598c8e979e8c5632aa61d7d23db7d5c0/lm_eval/api/task.py#L446
def n_shot(ds, n=10, seed=42):
  rnd = random.Random(seed)
  examples_indices = rnd.sample(range(len(ds)), n)
  examples = ds[examples_indices]

  gold_choices = np.array(examples["choices"])[np.arange(n), examples["gold"]]
  gold_choices_arr = np.char.array(gold_choices)
  query_char_arr = np.char.array(examples["query"])
  examples_strs = (query_char_arr + " " + gold_choices_arr).tolist()
  n_shot_examples = "\n\n".join(examples_strs)

  return n_shot_examples

In [9]:
n_shot(train_ds_process)

"Making a sandwich: Several food items and dishes are laid out on a table. Meat product and other items are used to create a sandwich. Then a bento shaper is used to create an image in the sandwich.\n\nThrowing darts: A man in a black vest is standing in a room. He throws darts at a dart board on the wall. A woman stands next to him watching.\n\nFood and Entertaining: How to make money by having a house party. Find a couple friends to help plan the party. Planning a killer house party is a lot easier when you've got accomplices to spread the time and responsibilities out to. Talk to your friends, and see who is interested in helping. This will make things a lot easier on you. Consider the following : Who has what responsibilities.\n\nHome and Garden: How to trim trees. Wear safety goggles, helmet, and purchase a step ladder. Safety goggles and a helmet or hard hat will protect your head and eyes as you prune adult trees. You may also need a small step ladder to reach higher branches. H

The below code runs on the validation dataset and cuts off after 100 iterations. This takes about 10 minutes on a T4 GPU.

In [ ]:
# Ref: https://github.com/couchette/gpt2_hellaswag_estimate
_ = model.eval()
correct = 0

for i, ex in tqdm(enumerate(validation_ds_process), leave=True, position=0, total=len(validation_ds_process)):
  n_shot_examples = n_shot(train_ds_process)

  scores = []
  for choice in ex["choices"]:
    prompt = n_shot_examples + "\n\n" + ex["query"] + ' ' + choice
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    with torch.no_grad():
      outputs = model(input_ids, labels=input_ids)
    loss = outputs.loss
    scores.append(loss.item())

  predicted_end = scores.index(min(scores))

  if predicted_end == ex["gold"]:
    correct += 1

  if i == 100: break

In [ ]:
print("accuracy normalized: ", str(correct) + "%")

accuracy normalized:  63%


👋 Conclusion

In this notebook I demonstrated how you can run simple hyperparameter finetuning sweeps with Phi-2 on the HellaSwag dataset. I hope you learned a thing or two about the finetuning process with Hugging Face and W&B! Thanks for reading! 👋😎